# Word processing

In [1]:
import json

In [9]:
filename = 'datasets/raw-wiktextract-data.jsonl'

In [47]:
with open(filename, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 8:
            break
        if line.strip():
            try:
                obj = json.loads(line)
                word = obj.get("word")
                pos = obj.get("pos")
                senses = obj.get("senses")
                sounds = obj.get("sounds")
                # categories = obj.get("categories")
                # topics = obj.get("topics")
                translations = obj.get("translations")
                
                
                print(f"\n\nWord: {word}")
                print(f"Pos: {pos}")
                # print(f" Senses: ")
                if senses:
                    definitions = list()
                    for item in senses:
                        if item.get('glosses'):
                            definitions += item["glosses"]
                    unique_definitions = set(definitions)
                    
                    print(f"Definitions: ")
                    for definition in unique_definitions:
                        print(f"- {definition}")
                                     
                print(f"Sounds: ")
                if sounds:
                    ipa = None
                    mp3_url = None
                    for item in sounds:
                        if ipa and mp3_url:
                            break
                        if ipa == None and item.get('ipa'):
                            ipa = item["ipa"]
                            print(f"ipa: {ipa}")
                            
                        if mp3_url == None and item.get('mp3_url'):
                            mp3_url = item["mp3_url"]
                            print(f"mp3_url: {mp3_url}")
                        
                if translations:
                    spanish_translation = None
                    for item in translations:
                        if spanish_translation:
                            break
                        if spanish_translation == None and item.get('lang'):
                            if(item["lang"] == "Spanish"):
                                spanish_translation = item["word"]
                                print(f"Translation spanish: {spanish_translation}")
                                
            except json.JSONDecodeError as e:
                print(f"Error decoding line {i + 1}: {e}")
                print(f"Line content: {line[:200]}")  # preview first 200 chars



Word: dictionary
Pos: noun
Definitions: 
- A synchronic dictionary of a standardised language held to only contain words that are properly part of the language.
- Any work that has a list of material organized alphabetically; e.g., biographical dictionary, encyclopedic dictionary.
- An associative array, a data structure where each value is referenced by a particular key, analogous to words and definitions in a dictionary (sense 1).
- A reference work with a list of words from one or more languages, normally ordered alphabetically, explaining each word's meanings (senses), and sometimes also containing information on its etymology, pronunciation, usage, semantic relations, and translations, as well as other data.
Sounds: 
ipa: /ˈdɪk.ʃə.nə.ɹi/
mp3_url: https://upload.wikimedia.org/wikipedia/commons/transcoded/1/1f/En-uk-dictionary.ogg/En-uk-dictionary.ogg.mp3
Translation spanish: diccionario


Word: dictionary
Pos: verb
Definitions: 
- To compile a dictionary.
- To add to a dictionary